In [1]:
from bs4 import BeautifulSoup
import requests
import random
import time
import numpy as np
import pandas as pd
import re

In [2]:
from selenium import webdriver
browser = webdriver.Firefox()

# UFC

In [3]:
browser.get('https://1xstavka.ru/line/ufc/2651172-ufc-fight-night-240324')
browser.implicitly_wait(2)
html = browser.page_source


# browser.quit()

In [4]:
bsObj = BeautifulSoup(html, 'lxml')
event_inf_tags = bsObj.findAll('div',{'class':'c-events__item c-events__item_game c-events-item'})


In [5]:
names = np.array([[re.sub(r'\(.*\)', '' , item.get_text()).strip() for item in event.find_all('span',{'class':'c-events__team'})] for event in event_inf_tags])
names

array([['Майлз Джонс', 'Коди Гибсон'],
       ['Мохаммед Усман', 'Майкл Паркин'],
       ['Игорь да Силва Северино', 'Андре Луис Лима Кардосо'],
       ['Монтсеррат Рендон', 'Дарья Железнякова'],
       ['Стивен Нгуен', 'Ярно Эрренс'],
       ['Рикардо Лукас Рамос', 'Джулиан Ероса'],
       ['Трей Огден', 'Курт Холобауг'],
       ['Билли Куарантилло', 'Юсеф Залал'],
       ['Карл Уильямс', 'Джастин Тафа'],
       ['Эдмен Шахбазян', 'ЭйДжей Добсон'],
       ['Луис Пахуэло', 'Фернандо Падилья'],
       ['Пэйтон Толбот', 'Кэмерон Саайман'],
       ['Аманда Рибас', 'Роуз Намаюнас']], dtype='<U23')

In [6]:
coefs = [[item.get_text().strip() for item in event.find_all('span',{'class':'c-bets__bet c-bets__bet_coef c-bets__bet_sm'})] for event in event_inf_tags]
coefs = np.array([[it[0], it[2]]  if len(it)>2 else [0,0] for it in coefs])
coefs

array([['1.68', '2.169'],
       ['2.126', '1.71'],
       ['2.408', '1.56'],
       ['2.725', '1.454'],
       ['1.51', '2.536'],
       ['1.59', '2.332'],
       ['1.686', '2.148'],
       ['1.71', '2.161'],
       ['1.553', '2.461'],
       ['1.553', '2.461'],
       ['2.287', '1.635'],
       ['1.67', '2.229'],
       ['2.998', '1.4']], dtype='<U5')

In [7]:
ar = np.concatenate([names, coefs], axis=1)
ar 

array([['Майлз Джонс', 'Коди Гибсон', '1.68', '2.169'],
       ['Мохаммед Усман', 'Майкл Паркин', '2.126', '1.71'],
       ['Игорь да Силва Северино', 'Андре Луис Лима Кардосо', '2.408',
        '1.56'],
       ['Монтсеррат Рендон', 'Дарья Железнякова', '2.725', '1.454'],
       ['Стивен Нгуен', 'Ярно Эрренс', '1.51', '2.536'],
       ['Рикардо Лукас Рамос', 'Джулиан Ероса', '1.59', '2.332'],
       ['Трей Огден', 'Курт Холобауг', '1.686', '2.148'],
       ['Билли Куарантилло', 'Юсеф Залал', '1.71', '2.161'],
       ['Карл Уильямс', 'Джастин Тафа', '1.553', '2.461'],
       ['Эдмен Шахбазян', 'ЭйДжей Добсон', '1.553', '2.461'],
       ['Луис Пахуэло', 'Фернандо Падилья', '2.287', '1.635'],
       ['Пэйтон Толбот', 'Кэмерон Саайман', '1.67', '2.229'],
       ['Аманда Рибас', 'Роуз Намаюнас', '2.998', '1.4']], dtype='<U23')

In [8]:
fighter_coef_df = pd.DataFrame(ar, columns=['fighter1', 'fighter2', 'coef1', 'coef2'])
fighter_coef_df

,fighter1,fighter2,coef1,coef2
0,Майлз Джонс,Коди Гибсон,1.68,2.169
1,Мохаммед Усман,Майкл Паркин,2.126,1.71
2,Игорь да Силва Северино,Андре Луис Лима Кардосо,2.408,1.56
3,Монтсеррат Рендон,Дарья Железнякова,2.725,1.454
4,Стивен Нгуен,Ярно Эрренс,1.51,2.536
5,Рикардо Лукас Рамос,Джулиан Ероса,1.59,2.332
6,Трей Огден,Курт Холобауг,1.686,2.148
7,Билли Куарантилло,Юсеф Залал,1.71,2.161
8,Карл Уильямс,Джастин Тафа,1.553,2.461
9,Эдмен Шахбазян,ЭйДжей Добсон,1.553,2.461


In [9]:
df_prev = pd.read_excel('coefs.xlsx')
# df_prev = pd.read_excel('coefs_ufc268.xlsx')



In [10]:
df_all = pd.concat([df_prev, fighter_coef_df], ignore_index=True)
df_all['coef1'] = df_all['coef1'].astype(float)

df_all['coef2'] = df_all['coef2'].astype(float)


df_all
# нули и пустоты удалить потом

,fighter1,fighter2,coef1,coef2
0,Карлос Вергара,Оде Осборн,2.600,1.520
1,Мелсик Багдасарян,Бруно Соуза,1.280,3.850
2,Дастин Джакоби,Джон Аллан Арте,1.250,4.150
3,Джан Вилланте,Крис Барнетт,1.780,2.080
4,Иан Гарри,Джордан Уильямс,1.250,4.100
...,...,...,...,...
762,Карл Уильямс,Джастин Тафа,1.553,2.461
763,Эдмен Шахбазян,ЭйДжей Добсон,1.553,2.461
764,Луис Пахуэло,Фернандо Падилья,2.287,1.635
765,Пэйтон Толбот,Кэмерон Саайман,1.670,2.229


In [11]:
df_all.to_excel('coefs.xlsx', index=False)

In [12]:
browser.quit()